In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import tensorflow as tf
sys.path.append("../")

In [2]:
import tensorflow as tf
from batcher import batcher
from pgn import PGN
from tqdm import tqdm

from utils.test_helper import beam_decode
from utils.config import CKPT_DIR
from utils.params import get_default_params
from utils.saveLoader import Vocab

In [3]:
from utils.test import test, test_and_save
from utils.test_helper import greedy_decode

In [23]:
params = get_default_params()
params["mode"] = "test"
params["batch_size"] = 4
params["num_to_test"] = 2

# 测试贪心预测

In [24]:
model = PGN(params)
print("Creating the checkpoint manager")
checkpoint = tf.train.Checkpoint(Seq2Seq=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, CKPT_DIR, max_to_keep=5)
checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Creating the checkpoint manager
Initializing from scratch.


In [18]:
checkpoint.restore(os.path.join(CKPT_DIR,"ckpt-2"))

In [25]:
vocab = Vocab(params["vocab_path"], params["vocab_size"])
ds = batcher(vocab, params)
batch = next(iter(ds))

mode=test


In [26]:
res = greedy_decode(model, batch, vocab, params)
res

['， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

以下是探究过程

In [7]:
# 初始化mask
start_index = vocab.word_to_id(vocab.START_DECODING)
stop_index = vocab.word_to_id(vocab.STOP_DECODING)
unk_index = vocab.word_to_id(vocab.UNKNOWN_TOKEN)

batch_size = params["batch_size"]  # 一个一个预测

测试集输入x

In [12]:
# enc_input shape (batch_size, enc_len)
enc_input = batch[0]["enc_input"]

# enc_output shape (batch_size, enc_len, enc_unit)
# enc_hidden shape (batch_size, enc_unit)
enc_output, enc_hidden = model.call_encoder(enc_input)  # update

# dec_input shape (batch_size, 1)
# dec_hidden shape (batch_size, enc_unit)
dec_input = tf.expand_dims([start_index] * batch_size, 1)  # update
dec_hidden = enc_hidden  # update

# enc_extended_inp shape (batch_size, enc_len)
# batch_oov_len shape (, )
# enc_pad_mask shape (batch_size, enc_len)
enc_extended_inp = batch[0]["extended_enc_input"]  # constant
batch_oov_len = batch[0]["max_oov_len"]  # constant
enc_pad_mask = batch[0]["sample_encoder_pad_mask"]  # constant
prev_coverage = None  # update

In [25]:
# 遍历步数
predicts=[''] * batch_size
steps = 0  # initial ste
while steps < params['max_dec_len']:
    final_preds, dec_hidden, context_vector, \
    attention_weights, p_gens, coverage_ret = model.call_decoder_onestep(dec_input,  # update
                                                                        dec_hidden,  # update
                                                                        enc_output,  # constant
                                                                        enc_extended_inp,  # constant
                                                                        batch_oov_len,  # constant
                                                                        enc_pad_mask,  # constant
                                                                        use_coverage=True,  # constant
                                                                        prev_coverage=prev_coverage  # update
                                                                        )
    predicted_ids = tf.argmax(final_preds, axis=2).numpy()
    dec_input = tf.cast(predicted_ids, dtype=dec_input.dtype)  # update
    prev_coverage = coverage_ret  # update
    
    for index,pred_id in enumerate(predicted_ids):
        # exmp: pred_id [458]
        pred_id = int(pred_id)
        if pred_id < vocab.count:
            predicts[index]+= vocab.id2word[pred_id] + ' '
        else:
            # if pred_id is oovs index
            predicts[index]+= batch[0]["article_oovs"][pred_id-vocab.count]
    steps += 1
    # print(steps)
    
results=[]
for predict in predicts:
    # 去掉句子前后空格
    predict=predict.strip()
    # 句子小于max len就结束了 截断
    if '<STOP>' in predict:
        # 截断stop
        predict=predict[:predict.index('<STOP>')]
    # 保存结果
    results.append(predict)

pred_id= 38 1
pred_id= 0 2
pred_id= 38 3
pred_id= 0 4
pred_id= 38 5
pred_id= 0 6
pred_id= 38 7
pred_id= 0 8
pred_id= 38 9
pred_id= 0 10
pred_id= 38 11
pred_id= 0 12
pred_id= 38 13
pred_id= 0 14
pred_id= 38 15
pred_id= 0 16
pred_id= 38 17
pred_id= 0 18
pred_id= 38 19
pred_id= 0 20
pred_id= 38 21
pred_id= 0 22
pred_id= 38 23
pred_id= 0 24
pred_id= 38 25
pred_id= 0 26
pred_id= 38 27
pred_id= 0 28
pred_id= 38 29
pred_id= 0 30
pred_id= 38 31
pred_id= 0 32
pred_id= 38 33
pred_id= 0 34
pred_id= 38 35
pred_id= 0 36
pred_id= 38 37
pred_id= 0 38
pred_id= 38 39
pred_id= 0 40
pred_id= 38 41
pred_id= 0 42
pred_id= 38 43
pred_id= 0 44
pred_id= 38 45
pred_id= 0 46
pred_id= 38 47
pred_id= 0 48
pred_id= 38 49
pred_id= 0 50
pred_id= 38 51
pred_id= 0 52


In [17]:
results

['机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ，',
 '机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ，',
 '机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ， 机油 ，']

# 测试beam_decode

In [6]:
params = get_default_params()

In [11]:
gen = test(params)
hy = next(gen)
hy.abstract

自己测试

In [23]:
test_and_save(params)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Building the model ...
Creating the vocab ...
Creating the batcher ...
Creating the checkpoint manager
Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\training_checkpoints\test_model-1
Model restored
mode=test


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.32s/it]
